In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="5"

os.chdir("/home/tugn232/projects/shap-e/")

In [2]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
from shap_e.util.image_util import load_image

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
xm = load_model('transmitter', device=device)
model = load_model('image300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

In [6]:
batch_size = 4
guidance_scale = 3.0

# To get the best result, you should remove the background and show only the object of interest to the model.
image = load_image("shap_e/examples/example_data/mug.jpg")

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(images=[image] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [8]:
render_mode = 'nerf' # you can change this to 'stf' for mesh rendering
size = 128 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAAOfn1+HgzdzbyNnZxNfZwtfWxdbVvtbTv9DSxtHTutDQv8/PvN…

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAAPf25PXz4PHw3Ofo1+Di0t3dy9nczNLczdjYxdTWw9PTv9PRwM…

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAAOHiz97fzN3ey9vcyNjaxdjYwtTXxdLWxNLTvtDTvs7TwNDQus…

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAAODbzNjYy9bVxdXSwdDRv9PPvdHPv9HPvM/PvNLOvdDOvdDOu9…